In [1]:
import os
from groq import Groq  
import json
from dotenv import load_dotenv

load_dotenv()

# Initialize Groq client with API key from .env
client = Groq(
    api_key=os.getenv("GROQ_API_KEY") 
)

In [ ]:
import serpapi
import os, requests

def get_search_result(query):
    client =serpapi.Client(api_key= os.getenv("SERP_API_KEY"))
    r= client.search({
        "q": query,
        "engine": "google",

    })

    results = ''
    for res in r['organic_results']:
        results += res['snippet'] + '\n'

    return (results)

def get_walmart_product_information(product_name):
    client =serpapi.Client(api_key= os.getenv("SERP_API_KEY"))
    r= client.search({
        "query": product_name,
        "engine": "walmart",
    })

    results = ""
    for res in r['organic_results']:
        results += 'Title: ' + res['title'] + '\t' + \
        'Rating:' + res['rating'] + '\t' + \
        'Out of Stock:' + str(res['in_stock']) + '\t' + \
        'Price:' + str(res['primary_offer']['offer_price']) + '\n\n'
    
    return results

def get_flight_booking_info(departure_id, arrival_id, outbound_date, return_date):
    client =serpapi.Client(api_key= os.getenv("SERP_API_KEY"))
    r = client.search({
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": "USD",
    })

    return r.get('best_flights', r['other_flights'])

In [4]:
# Test the function
print(get_search_result('Arsenal FC'))

HTTPError: 401 Client Error: Unauthorized for url: https://serpapi.com/search?q=Arsenal+FC&engine=google&api_key=23f1ad4252211fa299c2b89e1b4f424468a7012f

In [ ]:

# Define tools schema properly
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "Perform web search to answer questions about current events, people, or general knowledge",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_walmart_product_information",
            "description": "Get product details from Walmart",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "Name of product to search for"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_flight_booking_info",
            "description": "Search for flight information",
            "parameters": {
                "type": "object",
                "properties": {
                    "departure_id": {"type": "string", "description": "IATA code for departure airport"},
                    "arrival_id": {"type": "string", "description": "IATA code for arrival airport"},
                    "outbound_date": {"type": "string", "description": "Departure date (YYYY-MM-DD)"},
                    "return_date": {"type": "string", "description": "Return date (YYYY-MM-DD)"}
                },
                "required": ["departure_id", "arrival_id", "outbound_date"]
            }
        }
    }
]



In [ ]:
from datetime import date
import json
from groq import Groq 

# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

today = date.today()
today = today.strftime("%B %d, %Y")

messages = [
    {
        "role": "system",
        "content": f"""You are an assistant that can perform question answering with web search, 
        product information retrieval, and flight booking information retrieval.
        Today's date is: {today}"""
    },
    {
        "role": "user", 
        "content": "what happened to Jimmy Carter"
    }
]

# Corrected API call (chat.completions.create instead of chat_completion.create)
response = client.chat.completions.create(
    model="llama3-70b-8192",  # Correct model name
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=4006,
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(f"Initial Response: \n{response_message}\n\n")  # Fixed f-string

if tool_calls:
    available_functions = {
        "get_search_result": get_search_result,
        "get_walmart_product_information": get_walmart_product_information,
        "get_flight_booking_info": get_flight_booking_info
    }
    messages.append(response_message)

    for tool_call in tool_calls:
        function_name = tool_call.function.name  # Correct attribute
        function_to_call = available_functions.get(function_name)
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": str(function_response),
        })

    second_response = client.chat.completions.create(
        model="llama3-70b-8192",  # Correct model name
        messages=messages
    )
    response = second_response

print(f"Final Response: {response.choices[0].message.content}\n\n")

In [ ]:
print(os.getenv("SERPER_API_KEY"))

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("SERPER_API_KEY"))  # Verify it loads correctly


23f1ad4252211fa299c2b89e1b4f424468a7012f


In [6]:
from crewai_tools import SerperDevTool

# Initialize the search tool
tool = SerperDevTool()

def get_search_result(query):
    result = tool.run(query)  # Directly runs the query
    return result

# Test the function
print(get_search_result('Arsenal FC'))


/Users/a/Documents/DivverseLabs/AgenticAI_proj4/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


ModuleNotFoundError: No module named 'databricks'

In [7]:
from crewai_tools import SerperDevTool
tool = SerperDevTool()
print(tool.run("Arsenal FC latest news"))


/Users/a/Documents/DivverseLabs/AgenticAI_proj4/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/Users/a/Documents/DivverseLabs/AgenticAI_proj4/venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/a/Documents/DivverseLabs/AgenticAI_proj4/venv/lib/python3.10/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migra

Using Tool: Search the internet with Serper


TypeError: SerperDevTool._run() takes 1 positional argument but 2 were given

In [10]:
result = tool._run(search_query="Arsenal FC latest news")
print(result)


{'searchParameters': {'q': 'Arsenal FC latest news', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Arsenal News & Transfer Rumours - NewsNow', 'link': 'https://www.newsnow.com/us/Sports/Soccer/Premier+League/Arsenal', 'snippet': "Arsenal make 'final offer' worth £101m as promising update from 'those close to the player' emerges · Exclusive: Andrea Berta is interested in bringing £84m ...", 'position': 1}, {'title': 'Arsenal News: Latest News, Highlights & Club Updates', 'link': 'https://www.arsenal.com/news', 'snippet': 'Get all the breaking Arsenal news, live club updates and highlight videos from the official home of Arsenal. Sign up for exclusive video content.', 'position': 2, 'sitelinks': [{'title': 'Injury news on Nwaneri, Saka...', 'link': 'https://www.arsenal.com/news/injury-news-nwaneri-saka-martinelli-and-white'}, {'title': 'Team news: Lewis-Skelly...', 'link': 'https://www.arsenal.com/news/team-news-lewis-skelly-starts-champions-league'}, {'title':